## Basic app to extract from a ChatMessage the song and artist a user wants to play

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

Define your extraction goal (called "the response schema")

In [3]:
from langchain.output_parsers import ResponseSchema

In [4]:
##Tell LangChain what we are going to extract from the text

response_schemas = [
    ResponseSchema(
        name="singer",
        description="name of the singer"
    ),
    ResponseSchema(
        name="song",
        description="name of the song"
    )
]

**Create the Output Parser that will extract the data**
* To shape the response we receive from ChatGPT

In [6]:
from langchain.output_parsers import StructuredOutputParser

In [7]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

**Create the format instructions**

In [10]:
# Apply instructions to the output parser
format_instructions = output_parser.get_format_instructions()

In [11]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"singer": string  // name of the singer
	"song": string  // name of the song
}
```


**Create the ChatPromptTemplate**

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import HumanMessagePromptTemplate

In [15]:
#instance of chat model 
chat_model = ChatOpenAI()

In [17]:
template = """
Given a command from the user,
extract the artist and song names
{format_instructions}
{user_prompt}
"""

In [19]:
# Create the prompt template that we are going to use
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)
    ],
    input_variables={"user_prompt"},
    partial_variables={"format_instructions": format_instructions}
)

**Enter the chat message from the user**

In [20]:
user_message = prompt.format_prompt(
    user_prompt="I like the song Coraline by Måneskin"
)

In [21]:
user_chat_message = chat_model(user_message.to_messages())

**Extract singer and song from the user message**

In [22]:
extraction = output_parser.parse(user_chat_message.content)

In [23]:
print(extraction)

{'singer': 'Måneskin', 'song': 'Coraline'}


In [24]:
print(type(extraction))

<class 'dict'>
